In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import openpyxl
from openpyxl_image_loader import SheetImageLoader

In [3]:
xl = pd.ExcelFile("Data/eksisting Jalan Siliwangi.xlsx")

print("Sheet names: ", xl.sheet_names)

Sheet names:  ['RAMBU', 'PJU', 'RPPJ', 'MARKA', 'WARNING LIGHT']


In [4]:
for sheet in xl.sheet_names:
    df = xl.parse(sheet, header=1)
    print(f"\nSheet:", {sheet})
    print(df.head)


Sheet: {'RAMBU'}
<bound method NDFrame.head of    Unnamed: 0     Unnamed: 1   Unnamed: 2                       Unnamed: 3  \
0          NO  DETAIL LOKASI  DOKUMENTASI                      JENIS RAMBU   
1         NaN            NaN          NaN                         LARANGAN   
2         NaN            NaN          NaN  Jika Ya isi 1, Jika Tidak isi 0   
3           1            NaN          NaN                                1   
4           2            NaN          NaN                              NaN   
..        ...            ...          ...                              ...   
77         72            NaN          NaN                              NaN   
78         73            NaN          NaN                              NaN   
79         74            NaN          NaN                              NaN   
80         75            NaN          NaN                              NaN   
81         76            NaN          NaN                              NaN   

    Unnamed: 4 

In [15]:
df_temp = xl.parse("RAMBU", header=None)
table_name = df_temp.iloc[0].dropna().values[0]
print(table_name)

df_rambu = xl.parse("RAMBU", header=[2,3,4])

df_rambu.columns = ["_".join([str(c) for c in col if "Unnamed" not in str(c)]).strip() for col in df_rambu.columns.values]

df_rambu = df_rambu.loc[:, ~df_rambu.columns.str.contains("Rekap", case=False, na=False)]

df_rambu = df_rambu.dropna(how="all", subset=df_rambu.columns[1:])

df_rambu["DETAIL LOKASI"] = df_rambu["DETAIL LOKASI"].fillna(table_name)

binary_columns = [col for col in df_rambu.columns if col.startswith(("JENIS RAMBU", "LOKASI PEMASANGAN"))]
print(binary_columns)
df_rambu[binary_columns] = df_rambu[binary_columns].fillna("No").replace({1.0: "Yes"})

# Ensure "DOKUMENTASI" column exists and is of type str
if "DOKUMENTASI" not in df_rambu.columns:
    df_rambu["DOKUMENTASI"] = ""

df_rambu["DOKUMENTASI"] = df_rambu["DOKUMENTASI"].astype(str)

# Define folder to store images
output_folder = "extracted_images"
os.makedirs(output_folder, exist_ok=True)

pxl_doc = openpyxl.load_workbook("Data/eksisting Jalan Siliwangi.xlsx", data_only=True)
sheet_name = "RAMBU"
sheet = pxl_doc[sheet_name]

# Initialize image loader
image_loader = SheetImageLoader(sheet)
# Loop through rows and check for images in "DOKUMENTASI" column
dokumentasi_column = "C"  # Replace with the actual column letter
for row in range(6, sheet.max_row + 1):  # Start from data row
    cell_address = f"{dokumentasi_column}{row}"

    if image_loader.image_in(cell_address):
        # image = image_loader.get(cell_address)
        image = image_loader.get(cell_address).convert("RGB")  # Convert RGBA to RGB

        # Systematic naming: "tabname_row.jpg"
        image_name = f"{sheet_name}_{row}.jpg"
        image_path = os.path.join(output_folder, image_name)

        # Save the image
        image.save(image_path)

        # Update DataFrame (assuming df_rambu exists)
        df_rambu.loc[row - 6, "DOKUMENTASI"] = image_path

# Drop columns safely
df_rambu = df_rambu.dropna(axis=1, how="all")

print(df_rambu.head)

Ruas Jalan Siliwangi (Kuningan)
['JENIS RAMBU_LARANGAN_Jika Ya isi 1, Jika Tidak isi 0', 'JENIS RAMBU_PERINGATAN', 'JENIS RAMBU_PERINTAH', 'JENIS RAMBU_PETUNJUK', 'LOKASI PEMASANGAN_KIRI_Jika Ya isi 1, Jika Tidak isi 0', 'LOKASI PEMASANGAN_TENGAH', 'LOKASI PEMASANGAN_KANAN']
<bound method NDFrame.head of       NO                    DETAIL LOKASI                    DOKUMENTASI  \
0    1.0  Ruas Jalan Siliwangi (Kuningan)   extracted_images\RAMBU_6.jpg   
1    2.0  Ruas Jalan Siliwangi (Kuningan)   extracted_images\RAMBU_7.jpg   
2    3.0  Ruas Jalan Siliwangi (Kuningan)   extracted_images\RAMBU_8.jpg   
3    4.0  Ruas Jalan Siliwangi (Kuningan)   extracted_images\RAMBU_9.jpg   
4    5.0  Ruas Jalan Siliwangi (Kuningan)  extracted_images\RAMBU_10.jpg   
5    6.0  Ruas Jalan Siliwangi (Kuningan)  extracted_images\RAMBU_11.jpg   
6    7.0  Ruas Jalan Siliwangi (Kuningan)  extracted_images\RAMBU_12.jpg   
7    8.0  Ruas Jalan Siliwangi (Kuningan)  extracted_images\RAMBU_13.jpg   
8    9.0  